In [1]:
from API_KEYS import GENIUS_BEARER, GENIUS_CLIENT_ID, GENIUS_CLIENT_SECRET

In [2]:
print(list(map(lambda x: len(x), [GENIUS_BEARER, GENIUS_CLIENT_ID, GENIUS_CLIENT_SECRET])))

[64, 64, 86]


In [3]:
import requests
from bs4 import BeautifulSoup
import os
import re

In [4]:
# Get artist object from Genius API
def request_artist_info(artist_name, page=1):
    base_url = 'https://api.genius.com'
    headers = {'Authorization': 'Bearer ' + GENIUS_BEARER}
    search_url = base_url + '/search?per_page=10&page=' + str(page)
    data = {'q': artist_name}
    response = requests.get(search_url, data=data, headers=headers)
    return response
# Get Genius.com song url's from the artist
def request_song_url(artist_name, song_cap):
    page = 1
    songs = []

    while True:
        response = request_artist_info(artist_name, page)
        json = response.json()
        # Collect up to song_cap song objects from artist
        song_info = []
        for hit in json['response']['hits']:
            if artist_name.lower() in hit['result']['primary_artist']['name'].lower():
                song_info.append(hit)

        # Collect song URL's from song objects
        for song in song_info:
            if (len(songs) < song_cap):
                url = song['result']['url']
                songs.append(url)

        if (len(songs) == song_cap):
            break
        else:
            page += 1

    print('Found {} songs by {}'.format(len(songs), artist_name))
    return songs


# def request_song_urls(artist_name, song_cap=int(1e3)):
#     page = 1
#     songs = []
#     while True:
#         songs_info = []
#         response = request_artist_info(artist_name, page).json()
#         hits = response['response']['hits']
#         for hit in hits:
#             if artist_name.lower() in hit['result']['primary_artist']['name'].lower():
#                 songs_info.append(hit)
#         for song in songs_info:
#             if len(songs) < song_cap:
#                 url = song['result']['url']
#                 songs.append(url)
#             if (len(songs) == song_cap):
#                 break
#         else:
#             page += 1
#
#     print('Found {} songs by {}'.format(len(songs), artist_name))
#     return songs



In [8]:
# nonexistent artist, what's the response then
output = request_artist_info('Otsochodziiii',1)
if output.json()['response']['hits']:
    print('ok')

In [41]:
output = request_artist_info('Otsochodzi',1)

output.json()['response']

{'hits': [{'highlights': [],
   'index': 'song',
   'type': 'song',
   'result': {'annotation_count': 9,
    'api_path': '/songs/3313777',
    'artist_names': 'Otsochodzi',
    'full_title': 'Nie, nie by\xa0Otsochodzi',
    'header_image_thumbnail_url': 'https://images.genius.com/065c15c04b36c65b669d85950970989e.300x300x1.jpg',
    'header_image_url': 'https://images.genius.com/065c15c04b36c65b669d85950970989e.640x640x1.jpg',
    'id': 3313777,
    'language': 'pl',
    'lyrics_owner_id': 291569,
    'lyrics_state': 'complete',
    'path': '/Otsochodzi-nie-nie-lyrics',
    'pyongs_count': 23,
    'relationships_index_url': 'https://genius.com/Otsochodzi-nie-nie-sample',
    'release_date_components': {'year': 2017, 'month': 11, 'day': 15},
    'release_date_for_display': 'November 15, 2017',
    'release_date_with_abbreviated_month_for_display': 'Nov. 15, 2017',
    'song_art_image_thumbnail_url': 'https://images.genius.com/065c15c04b36c65b669d85950970989e.300x300x1.jpg',
    'song_art

In [44]:
request_song_url('Lana Del Rey', 3)

Found 3 songs by Lana Del Rey


['https://genius.com/Lana-del-rey-young-and-beautiful-lyrics',
 'https://genius.com/Lana-del-rey-summertime-sadness-lyrics',
 'https://genius.com/Lana-del-rey-love-lyrics']

In [45]:
def scrape_song_lyrics(url):
    page = requests.get(url)
    html = BeautifulSoup(page.text, 'html.parser')
    lyrics = html.find('div', class_=re.compile("^lyrics$|Lyrics__Root")).get_text()
    #remove identifiers like chorus, verse, etc
    lyrics = re.sub(r'[\(\[].*?[\)\]]', '', lyrics)
    #remove empty lines
    lyrics = os.linesep.join([s for s in lyrics.splitlines() if s])
    return lyrics


In [47]:
print(scrape_song_lyrics('https://genius.com/Lana-del-rey-young-and-beautiful-lyrics'))

AttributeError: 'NoneType' object has no attribute 'get_text'